**EXLlamaV2 Quantization**

Notebook by UnstableLlama


---




[ExLlamaV2 Repo](https://github.com/turboderp/exllamav2)

---



Find an unquantized model repo in either pytorch_model.bin or model.safetensors format, copy the url and paste it in the cell below at the comment

In [ ]:
!pip install git-lfs
!git lfs install
!git clone https://huggingface.co/ehartford/Samantha-1.11-7b # Replace with your model repo URL
!git clone https://github.com/turboderp/exllamav2/

Git LFS initialized.
Cloning into 'Samantha-1.11-7b'...
remote: Enumerating objects: 19, done.
remote: Total 19 (delta 0), reused 0 (delta 0), pack-reused 19
Unpacking objects: 100% (19/19), 482.12 KiB | 1.07 MiB/s, done.
Filtering content: 100% (3/3), 4.55 GiB | 8.96 MiB/s, done.
Encountered 1 file(s) that may not have been copied correctly on Windows:
	pytorch_model-00001-of-00002.bin

See: `git lfs help smudge` for more details.
Cloning into 'exllamav2'...
remote: Enumerating objects: 732, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 732 (delta 107), reused 118 (delta 89), pack-reused 569
Receiving objects: 100% (732/732), 1.20 MiB | 4.08 MiB/s, done.
Resolving deltas: 100% (457/457), done.


Copy the path of your model folder from the file navigator and paste below

In [ ]:
directory = "/content/Samantha-1.11-7b"   # Replace with the path to your model, from the file directory to the left

In [ ]:
!pip install safetensors
!pip install ninja
!pip install sentencepiece
!pip install fastparquet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.6 MB/s eta 0:00:00


In [ ]:
%mkdir /content/quants
%cd exllamav2
!wget -O 0000.parquet "https://huggingface.co/datasets/WizardLM/WizardLM_evol_instruct_70k/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet"

/content/exllamav2
--2023-09-23 07:39:35--  https://huggingface.co/datasets/WizardLM/WizardLM_evol_instruct_70k/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet
Resolving huggingface.co (huggingface.co)... 13.35.166.36, 13.35.166.114, 13.35.166.69, ...
Connecting to huggingface.co (huggingface.co)|13.35.166.36|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/b1/cc/b1cc601ca4564f8c8627fd262f574a67c7a16a521b3c2dcf7ceda8424f911360/3f96119d72a1469b19640c5d24333f1beaadeec6fdc42be2b7f5023a03e89f9d?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%270000.parquet%3B+filename%3D%220000.parquet%22%3B&Expires=1695713975&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5NTcxMzk3NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9iMS9jYy9iMWNjNjAxY2E0NTY0ZjhjODYyN2ZkMjYyZjU3NGE2N2M3YTE2YTUyMWIzYzJkY2Y3Y2VkYTg0MjRmOTExMzYwLzNmOTYxMTlkNzJhMTQ2OWIxOTY0



---


**Convert to safetensors**

Only run these next two cells
if your model is in pytorch_model-00001-of-0000X.bin format


---



In [ ]:
import os
import re
import torch
from safetensors.torch import load_file, save_file

# Function to check file size
def check_file_size(sf_filename: str, pt_filename: str):
    sf_size = os.stat(sf_filename).st_size
    pt_size = os.stat(pt_filename).st_size
    if (sf_size - pt_size) / pt_size > 0.01:
        raise RuntimeError(
            f"""The file size difference is more than 1%:
         - {sf_filename}: {sf_size}
         - {pt_filename}: {pt_size}
         """
        )

# Function to convert individual file
def convert_file(pt_filename: str, sf_filename: str):
    loaded = torch.load(pt_filename, map_location="cpu")
    if "state_dict" in loaded:
        loaded = loaded["state_dict"]
    loaded = {k: v.contiguous() for k, v in loaded.items()}
    os.makedirs(os.path.dirname(sf_filename), exist_ok=True)
    save_file(loaded, sf_filename, metadata={"format": "pt"})
    check_file_size(sf_filename, pt_filename)
    reloaded = load_file(sf_filename)
    for k in loaded:
        pt_tensor = loaded[k]
        sf_tensor = reloaded[k]
        if not torch.equal(pt_tensor, sf_tensor):
            raise RuntimeError(f"The output tensors do not match for key {k}")

def convert_all_files_in_directory(directory: str):
    for filename in os.listdir(directory):
        pt_filename = os.path.join(directory, filename)
        sf_filename = None  # Initialize to None, will be set later if a match is found

        # For files matching "pytorch_model-(\d+)-of-(\d+).bin"
        match = re.match(r"pytorch_model-(\d+)-of-(\d+).bin", filename)
        if match:
            part_num, total_parts = match.groups()
            sf_filename = os.path.join(directory, f"model-{part_num.zfill(5)}-of-{total_parts.zfill(5)}.safetensors")

        # For files matching "pytorch_model.bin"
        elif filename == "pytorch_model.bin":
            sf_filename = os.path.join(directory, "model.safetensors")

        # If a match was found, convert the file
        if sf_filename:
            convert_file(pt_filename, sf_filename)

if __name__ == "__main__":
    directory = "/home/unstable/LLM/exllamav2/AI/OO-Mistral 7b"  # Replace with your directory path
    convert_all_files_in_directory(directory)

Deletes the pytorch_model.bin files to free up space after conversion

---



In [ ]:
import os
import re

# Function to delete all .bin files in a directory
def delete_all_bin_files_in_directory(directory: str):
    for filename in os.listdir(directory):
        match = re.match(r"pytorch_model-(\d+)-of-(\d+).bin", filename)
        if match:
            file_path = os.path.join(directory, filename)
            os.remove(file_path)
            print(f"Deleted {file_path}")

# Run the deletion
delete_all_bin_files_in_directory(directory)

Deleted /content/Samantha-1.11-7b/pytorch_model-00001-of-00002.bin
Deleted /content/Samantha-1.11-7b/pytorch_model-00002-of-00002.bin


**Quantize**
---
[Documentation here](https://github.com/turboderp/exllamav2/blob/master/doc/convert.md)

---

Choose your BPW below at comment

In [ ]:
import torch
import subprocess

input_dir = directory
bpw = "4"                                # replace with desired bpw
output_dir = "/content/quants"
calibr_file = "/content/exllamav2/0000.parquet"

def run_command_and_stream_output(command):
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print(output.strip())

command = [
    "python",
    "convert.py",
    "-i", input_dir,
    "-o", output_dir,
    "-c", calibr_file,
    "-b", bpw
]

run_command_and_stream_output(command)

Streaming output truncated to the last 5000 lines.
-- 1.0:6b 32g s4                  6.13 bpw    rfn_error: 0.01492
-- 0.1:8b/0.9:6b 128g s4          6.22 bpw    rfn_error: 0.01460
-- 1.0:8b 32g s4                  8.13 bpw    rfn_error: 0.00938
-- Time: 5.18 seconds
-- Linear: model.layers.7.self_attn.o_proj
-- 0.05:3b/0.95:2b 32g s4         2.19 bpw    rfn_error: 0.21073
-- 0.25:3b/0.75:2b 32g s4         2.38 bpw    rfn_error: 0.18893
-- 0.25:4b/0.75:2b 32g s4         2.63 bpw    rfn_error: 0.17931
-- 0.1:4b/0.4:3b/0.5:2b 32g s4    2.72 bpw    rfn_error: 0.15321
-- 0.1:4b/0.9:3b 32g s4           3.22 bpw    rfn_error: 0.09953
-- 0.2:6b/0.8:3b 32g s4           3.69 bpw    rfn_error: 0.09055
-- 1.0:3b 128g s4                 3.03 bpw    rfn_error: 0.11910
-- 1.0:3b 32g s4                  3.13 bpw    rfn_error: 0.10821
-- 0.05:4b/0.95:3b 32g s4         3.19 bpw    rfn_error: 0.10178
-- 0.4:4b/0.6:3b 32g s4           3.53 bpw    rfn_error: 0.08330
-- 0.6:4b/0.4:3b 64g s4           3.66 

**Save the files**

---

Mounts google drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Renames all output(s).safetensors to model(s).safetensors and copies to a new folder in your google drive named "Model"

In [ ]:
!mkdir -p /content/drive/MyDrive/Model
!for f in /content/quants/output*.safetensors; do
  new_name=$(echo $f | sed 's/output-/model-/')
  mv $f $new_name
  cp $new_name /content/drive/MyDrive/Model
done
!cp /content/quants/measurement.json /content/drive/MyDrive/Model

mv: cannot stat '/content/quants/output.safetensors': No such file or directory
cp: cannot stat '/content/quants/measurement.json': No such file or directory
cp: cannot stat '/content/quants/model.safetensors': No such file or directory


Copies the rest of the model config files to your new google drive /Model folder

In [ ]:
import os
import shutil

# Function to copy all files except those with specified prefixes or suffixes
def copy_files_exclude(src_dir: str, dest_dir: str):
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    for filename in os.listdir(src_dir):
        if not (filename.endswith('.safetensors') or
                filename.startswith('pytorch') or
                filename.startswith('.')):  # Skip hidden files/folders
            src_file_path = os.path.join(src_dir, filename)
            dest_file_path = os.path.join(dest_dir, filename)

            if os.path.isdir(src_file_path):
                shutil.copytree(src_file_path, dest_file_path)
            else:
                shutil.copy2(src_file_path, dest_file_path)
            print(f"Copied {src_file_path} to {dest_file_path}")

# Define source and destination directories
src_dir = directory
dest_dir = /content/drive/MyDrive/Model

# Run the copy
copy_files_exclude(src_dir, dest_dir)


Copied /content/Samantha-1.11-7b/generation_config.json to /content/your_dest_directory_here/generation_config.json
Copied /content/Samantha-1.11-7b/tokenizer.model to /content/your_dest_directory_here/tokenizer.model
Copied /content/Samantha-1.11-7b/README.md to /content/your_dest_directory_here/README.md
Copied /content/Samantha-1.11-7b/tokenizer.json to /content/your_dest_directory_here/tokenizer.json
Copied /content/Samantha-1.11-7b/added_tokens.json to /content/your_dest_directory_here/added_tokens.json
Copied /content/Samantha-1.11-7b/config.json to /content/your_dest_directory_here/config.json
Copied /content/Samantha-1.11-7b/special_tokens_map.json to /content/your_dest_directory_here/special_tokens_map.json
Copied /content/Samantha-1.11-7b/tokenizer_config.json to /content/your_dest_directory_here/tokenizer_config.json


Name your quantized model below, include the original name and the BPW, end with -exl2

In [ ]:
modName = "Samantha-1.11-7b-4bpw-exl2"  # Replace with your quantized model's name


Renames the new /Model folder in your google drive

In [ ]:
import os

# Existing folder path
existing_folder_path = "/content/drive/MyDrive/Model"

# New folder path
new_folder_path = os.path.join(os.path.dirname(existing_folder_path), modName)

# Rename the folder
os.rename(existing_folder_path, new_folder_path)
print(f"Renamed folder to: {new_folder_path}")

Renamed folder to: /content/drive/MyDrive/Samantha-1.11-7b-4bpw-exl2


That's it! You now have a functioning quantized model in a folder that you can copy in to your favorite webui's /models directory for inference.